In [1]:
# sae_results/ae_layer12/nz_feature_ind.pt sae_results/ae_layer12/total_reward_diff.pt check_top_reward.ipynb
import torch
ae_name = 'ae_layer12'
nz_feature_ind = torch.load(f'sae_results/{ae_name}/nz_feature_ind.pt')
total_reward_diff = torch.load(f'sae_results/{ae_name}/total_reward_diff.pt')

In [2]:
total_reward_diff.shape, total_reward_diff[0]

(torch.Size([9180, 10]),
 tensor([ 9.5367e-07,  9.5367e-07,  5.9605e-07,  9.5367e-07,  9.5367e-07,
          1.9073e-06,  0.0000e+00,  2.8610e-06, -2.8610e-06, -1.9073e-06]))

In [21]:
pos_only = ((total_reward_diff > 0).sum(-1) == 10)
neg_only = ((total_reward_diff < 0).sum(-1) == 10)
tmp_reward = total_reward_diff.clone()
tmp_reward[~pos_only & ~neg_only] = 0
pos_only.sum(), neg_only.sum()

(tensor(993), tensor(134))

In [23]:
tmp_reward.sum(-1).topk(10), tmp_reward.sum(-1).topk(10, largest=False)

(torch.return_types.topk(
 values=tensor([47.1262, 37.1970, 20.3115, 15.5119, 14.2063, 14.1882, 13.2903, 12.4759,
         11.4827, 11.4094]),
 indices=tensor([9173, 5734, 8192,  340, 1998, 8466, 2082, 4854, 6811,  996])),
 torch.return_types.topk(
 values=tensor([-40.1438, -33.9029, -12.5370, -12.0577,  -7.9758,  -7.0178,  -6.1965,
          -6.0784,  -5.0111,  -4.9518]),
 indices=tensor([ 695, 6607, 4070, 6427, 1235, 3288, 4888, 5121,  476, 2243])))

In [24]:
top_ind = tmp_reward.sum(-1).topk(10).indices
low_ind = tmp_reward.sum(-1).topk(10, largest=False).indices
# pos_only_ind = pos_only[top_ind]
# neg_only_ind = neg_only[low_ind]
# pos_only_ind, neg_only_ind

In [20]:
top_ind, pos_only

(tensor([992, 613, 890,  29, 208, 918, 216, 512, 737,  93]),
 torch.Size([9180]))

In [25]:
nz_feature_ind[top_ind], nz_feature_ind[low_ind]

(tensor([32744, 20512, 29242,  1197,  6921, 30221,  7234, 17262, 24335,  3498]),
 tensor([ 2420, 23586, 14450, 22933,  4353, 11612, 17368, 18210,  1625,  7803]))

In [14]:
neg_only

tensor([False, False, False,  ..., False, False, False])

In [16]:
top_ind

torch.return_types.topk(
values=tensor([47.1262, 37.1970, 20.3115, 15.5119, 14.2063, 14.1882, 13.2903, 12.4759,
        11.4827, 11.4094]),
indices=tensor([992, 613, 890,  29, 208, 918, 216, 512, 737,  93]))